<a href="https://colab.research.google.com/github/KhoomeiK/MindMapResearch/blob/master/HAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from csv import reader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
class Users(Dataset):
    def __init__(self, path): # TODO: data reading
      docs = reader(path)
      # docs = [[[[infersentVector, ..., sentN], ..., postN], label0], ..., [userN, labelN]]
      for u, l in docs: # torchify
        self.users.append(u) # [[infersentVector, ..., sentN], ..., postN] (100 posts, each with arbitrary # vector sents)
        self.labels.append(l)
      
      print(len(self), self[0])

    def __getitem__(self, i):
      return self.users[i], self.labels[i]
    
    def __len__(self):
      assert len(self.users) == len(self.labels)
      return len(self.users)

In [0]:
class HAN(nn.Module):
    def __init__(
        self,
        batch_size=16,
        embedding_dimension=300, # from glove to infersent?
        hidden_size=150, 
        n_layers=1, # multiple?
    ):
        super(HAN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        '''
        GRUforward(sentences)
        GRUbackward(sentences)
        concat
        attention(each sentence state) # basically a post vector
        GRUforward(post vectors)
        # backward?
        attention(each post state)
        linear output
        '''

        self.gru1 = nn.GRU( # pass through sent vecs
            embedding_dimension,
            hidden_size,
            num_layers=n_layers,
            bidirectional=True
        )

        self.mlp1 = nn.Linear(hidden_size * 2, hidden_size) # attention mlp
        self.contextW1 = nn.Parameter(torch.Tensor(hidden_size * 2, 1)) # attention weights

        self.gru2 = nn.GRU( # pass through post vecs
            1, # 1 summed val for each post
            hidden_size,
            num_layers=n_layers,
            bidirectional=True
        )

        self.mlp2 = nn.Linear(hidden_size * 2, hidden_size) # attention mlp
        self.contextW2 = nn.Parameter(torch.Tensor(hidden_size * 2, 1)) # attention weights

        # self.classify = nn.Linear() ?
    
    def init_hidden(self):
        pass

    def forward(self, user):
        postVecs = torch.tensor([])
        for post in user:
            sentAnnot = self.gru1(post) # tensor of sentence annotations of post
            sentU = F.Tanh(self.mlp1(sentAnnot)) # hidden reps of sentAnnots
            sentA = F.Softmax(sentU * self.contextW1) # att weights of sents
            postVec = torch.sum(sentA * sentAnnot) # total val of post; 1 sum?
            torch.cat((postVecs, postVec)) 

        postAnnot = self.gru2(postVecs)
        postU = F.Tanh(self.mlp2(sentAnnot)) # hidden reps of postAnnots
        postA = F.Softmax(postU * self.contextW2) # att weights of posts
        userVec = torch.sum(postA * postAnnot) # total val of user; 1 sum?

        output = F.Softmax(userVec) # classify
        
        return output

In [0]:
# curl dataset
# init Users
# data = tqdm_notebook(DataLoader(), leave=False)

In [8]:
model = HAN().to(device)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()
losses = []
model.train()

for epoch in range(4):
    total = 0
    for X, Y in data:
        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred, Y)
        loss.backward()
        optimizer.step()

        total += 1
        losses.append(loss.item())
        data.set_description(f'Loss: {loss.item():.3f}')
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)

    tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')

NameError: ignored